In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sk

bike_daily = pd.read_csv('daily_weather.csv')
bike_master = pd.read_csv('bike_full.csv', dtype={5: str, 7: str})  # Convert both columns to strings



In [12]:
bike_master_clean = bike_master.copy()

# Convert datetime columns
bike_master_clean['started_at'] = pd.to_datetime(bike_master_clean['started_at'])
bike_master_clean['ended_at'] = pd.to_datetime(bike_master_clean['ended_at'])

# Convert category columns
category_columns = ['rideable_type', 'start_station_name', 'end_station_name', 'member_casual']
for col in category_columns:
    bike_master_clean[col] = bike_master_clean[col].astype('category')

# Convert string columns (IDs)
string_columns = ['ride_id', 'start_station_id', 'end_station_id']
for col in string_columns:
    bike_master_clean[col] = bike_master_clean[col].astype('string')



In [ ]:
bike_master_clean.dtypes
bike_master_clean.head()
null_percentages = (bike_master_clean.isnull().sum() / len(bike_master_clean)) * 100
bike_master_clean_no_nulls = bike_master_clean.dropna()
bike_master_clean_no_nulls.isnull().sum()
bike_master_clean_no_nu

Index([       0,        2,        3,        4,        6,        8,        9,
             10,       11,       12,
       ...
       10693984, 10693986, 10693987, 10693989, 10693990, 10693991, 10693992,
       10693993, 10693994, 10693995],
      dtype='int64', length=9572692)